In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt

import numpy as np

# Ex.1 - Stokes problem - patch test

\begin{equation*}
\begin{cases}
- \Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega=(0,3)\times(0,1), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
(\nabla\boldsymbol{u}-pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_1\cup\Gamma_2,\\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \Gamma_3, \\
\boldsymbol{u} = \boldsymbol{g}_\text{D} & {\rm on} \ \Gamma_4.
\end{cases}
\end{equation*}

with $\boldsymbol{g}_\text{D} = 1\boldsymbol{i}$.

In [ ]:
from firedrake import RectangleMesh

# Build the mesh
n = 10
mesh = RectangleMesh(3*n, n, 3, 1)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces

# P1-P0
V = VectorFunctionSpace(mesh, 'P', 1)
Q = FunctionSpace(mesh, 'DP', 0) # NB: P0 are DISCONTINUOUS elements (DP)

# P1-P1
# V = VectorFunctionSpace(mesh, 'P', 1)
# Q = FunctionSpace(mesh, 'P', 1)

# P1b-P1
# The enrichment of the velocity space has to be done at the finite element level
V1_el = FiniteElement('CG', mesh.ufl_cell(), 1)
B_el = FiniteElement('Bubble', mesh.ufl_cell(), mesh.topological_dimension() + 1)
V_el = VectorElement(NodalEnrichedElement(V1_el, B_el))
V = FunctionSpace(mesh, V_el)
Q = FunctionSpace(mesh, 'P', 1)

# P2-P1
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)

W = MixedFunctionSpace([V, Q])
print('Ndofs - velocity :',V.dim(),', pressure :',Q.dim(),', total :',W.dim())

# Finite element functions
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

In [ ]:
# Boundary conditions (strong)
bc3 = DirichletBC(W.sub(0), Constant((0., 0.)), 3)
bc4 = DirichletBC(W.sub(0), Constant((1., 0.)), 4)
bcs = (bc3, bc4)

# Variational formulation
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
L = inner(Constant((0.0,0.0)), v) * dx
  # Dummy rhs (=0) to ensure that the solve recognize a==L as a linear problem

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
# Variational formulation (penalty method)
eps = 1.e-30
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx \
    + 1./eps*inner(u,v)*ds(3) + 1./eps*inner(u,v)*ds(4)
L = 1./eps*inner(Constant((1.,0.)),v) * ds(4)

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

In [ ]:
# Error computation
x = SpatialCoordinate(mesh)
u_ex = as_vector([x[1],0.])
grad_u_ex = as_tensor([[0.,1.],[0.,0.]])
p_ex = Constant(0.)
errL2u = sqrt(assemble( inner(uh-u_ex,uh-u_ex) * dx ))
errH10u = sqrt(assemble( inner(grad(uh)-grad_u_ex,grad(uh)-grad_u_ex) * dx ))
errL2p = sqrt(assemble( inner(ph-p_ex,ph-p_ex) * dx ))
print('Errors - L2-u:', errL2u, ', H10-u:', errH10u, ', L2-p:', errL2p)

In [ ]:
mesh = RectangleMesh(3*n, n, 3, 1)
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])

bc1 = DirichletBC(W.sub(0), Constant((0., 0.)), 3)
bc3 = DirichletBC(W.sub(0), Constant((0., 0.)), 3)
bc4 = DirichletBC(W.sub(0), Constant((1., 0.)), 4)
bcs = (bc3, bc4)


In [ ]:
# Solve Stokes on (0,1)^2 with fully Dirichlet BC.
# Inputs: n         = mesh subdivisions along each direction
#         degreeU   = polynomial degree of velocity space
#         degreeU   = polynomial degree of velocity space
#         gD_fun    = Dirichlet data (generic function)
#         f_fun     = source term (generic function)
def solve_stokes(n, degreeU, degreeP, gD_fun, f_fun):
    # Mesh definition
    mesh = UnitSquareMesh(n, n, 'crossed')

    V = VectorFunctionSpace(mesh, 'P',degreeU)

    if degreeP==0:
        Q = FunctionSpace(mesh, 'DP', 0) #P0 element is discontinuous
    else:
        Q = FunctionSpace(mesh, 'P', degreeP)

    W = MixedFunctionSpace([V, Q])

    # The data generic expression are interpolated on the current mesh.
    x = SpatialCoordinate(mesh)
    gD = gD_fun(x)
    f = f_fun(x)








    bc = DirichletBC(W.sub(0),gD,(1,2,3,4))

    # Variational formulation
    u, p = TrialFunctions(W)
    v, q = TestFunctions(W)

    a = inner(grad(u),grad(v))*dx -div(v)*p*dx -div(u)*q*dx
    L = inner(f,v)*dx #for tensors

    # Solution
    #fully dirichlet ->nullspace is p=const

    nullsp=MixedVectorSpaceBasis(W,[W.sub(0),VectorSpaceBasis(constant=True)])   #to impose integral mean =0

    w = Function(W)
    solve( a==L,w,bcs=list(bc),nullspace=nullsp)
    u, p = w.split()

    return u, p, mesh

In [ ]:

n_vec = np.array((10, 20, 40))
err_u_L2 = np.zeros(n_vec.shape[0])
err_u_H1 = np.zeros(n_vec.shape[0])
err_p_L2 = np.zeros(n_vec.shape[0])
err_p_H1 = np.zeros(n_vec.shape[0])

# Exact solution and source term as general functions:
# to avoid mesh dependence, we define them as lambda functions.
u_ex_fun = lambda x: as_vector((
    -cos(x[0]) * sin(x[1]),
    sin(x[0]) * cos(x[1])))

p_ex_fun = lambda x: -1/4.0*(cos(2*x[0])+cos(2*x[1])) +(sin(2.0)/4.0)

f_fun = lambda x: as_vector((
    -2*cos(x[0])*sin(x[1])+ 1/2.0 *sin(2*x[0]),
    2*sin(x[0])*cos(x[1])+ 1/2.0 *sin(2*x[1]) ))

for ii in range(n_vec.shape[0]):
    n = n_vec[ii]
    uh, ph, mesh = solve_stokes(n, 2, 1, u_ex_fun, f_fun)

    # Interpolate lambda functions on current mesh
    x = SpatialCoordinate(mesh)
    u_ex =u_ex_fun(x)
    p_ex = p_ex_fun(x)

    err_u_L2[ii] = errornorm(u_ex, uh, 'L2')
    err_u_H1[ii] = errornorm(u_ex, uh, 'H1')
    err_p_L2[ii] = errornorm(p_ex, ph, 'L2')


    print('n = ', n_vec[ii])
    print('L2 error, velocity = ', err_u_L2[ii])
    print('H1 error, velocity = ', err_u_H1[ii])
    print('L2 error, pressure = ', err_p_L2[ii])

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')


#pressure high if you don't make Q=L0^2

In [ ]:
H=[1/n for n in n_vec]
print(H)
H=np.array(H)

In [ ]:


plt.figure(figsize=(8, 6))
plt.loglog(H, err_u_L2, marker='o', linestyle='-', color='b', label='ErrorL2 for v')
plt.loglog(H,err_u_H1, marker='o', linestyle='-', color='r', label='ErrorH1 for v')
plt.loglog(H,err_p_L2, marker='o', linestyle='-', color='g', label='ErrorH1 for p')

plt.loglog(H, H, marker='o', linestyle='--', color='g', label='H')
plt.loglog(H, H**2, marker='o', linestyle='--', color='y', label='H^2')
plt.xlabel('Mesh Size')
plt.title('Convergence Plot')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()

In [ ]:
varpb=LinearVariationalProblem(a,L,w,list(bc))

parameters={'ksp_type':'gmres',
            'ksp_rtol':1.e-5,
            'ksp_max_it':10000,
            'pc_type':'bjacobi'}

solver=LinearVariationalSolver(varpb,solver-paramters=params)
solver.solve()